In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame({'Year':[i for i in range(2012,2022)],
                     'State':['a','b','c','a','b','c','a','c','v','a'],
                     'sta_con':['a','a','a','a','a','a','a','b','a','a'],
                     'yield(t/ha)':[1,2,3,3,5,6,7,8,8.5,10]
                        })

data

In [ ]:
data = pd.read_csv('D:/论文-产量趋势利用/数据/input_soybean_exp.csv',engine='python',encoding="utf8",nrows = 500)
data[['Year','sta_con','Longitude','Latitude']]

In [ ]:
county = data.drop_duplicates(subset=['sta_con','Longitude','Latitude'])
county

In [ ]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
    Replicating the same formula as mentioned in Wiki
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km
def find_nearest(lat, long, N):
    distances = county.apply(
        lambda row: dist(lat, long, row['Latitude'], row['Longitude']), 
        axis=1)
    return list(county.loc[distances.nsmallest(n=N,keep = 'all').index, 'sta_con'])

In [ ]:
distances = county.apply(
    lambda row: dist(40.174752, -88.905468, row['Latitude'], row['Longitude']), 
    axis=1)
list(county.loc[distances.nsmallest(n=6,keep = 'all').index, 'sta_con'])

In [ ]:
data['name'] = data.apply(
    lambda row: find_nearest(row['Latitude'], row['Longitude'],5), 
    axis=1)
# To check the data frame if it has a new column of hotel name (for each and every member's location in the list)
data[['Year','sta_con','name','Longitude','Latitude']]

In [ ]:
def average_yield_of_N_years(data,N):
    for county in set(data['sta_con']):
        for year in range(2014,2022):
            yearlist = [i for i in range(year-N,year)]
            data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N5_EXP2'] = data[(data['Year'].isin(yearlist)) & (data['sta_con']==county)]['yield(t/ha)'].mean()
    data = data.fillna(data.groupby(['Year','State']).transform('mean'))
    return data

In [ ]:
average_yield_of_N_years(data,2)[['Year','sta_con','yield(t/ha)','yield_N5_EXP2']]

In [ ]:
set(data['State'])

In [ ]:
# 分州去趋势
test_year = [2020]
N = 5
trend_n = 10
data_test = data[data['Year'].isin(test_year)]
# tra-val set
data_train_val = data[data['Year'].isin([i for i in range(test_year[0]-N,test_year[0])])]
# split input and output
y_train_val = data_train_val['yield(t/ha)']
y_test = data_test['yield(t/ha)']
# de-trend the yield using linear model
yield_mean = data_train_val['yield(t/ha)'].mean()
linear_m = linear_model.LinearRegression()
#         if usehistoy:
#             X99 = np.array(data1980['year']).reshape(-1,1)
#             Y99 = np.array(data1980['yield(t/ha)']).reshape(-1,1)
#         else:
# X99 = np.array(data_train_val['Year']).reshape(-1,1)
# Y99 = np.array(data_train_val['yield(t/ha)']).reshape(-1,1)


count = 0
data_train_valnew1 = pd.DataFrame([],columns = data_train_val.columns)
data_testnew1 = pd.DataFrame([],columns = data_test.columns)
for i in ['illinois']:
    data_train_valnew = data_train_val[data_train_val['State'].isin([i])]
    data_testnew = data_test[data_test['State'].isin([i])]
    X99 = np.array(data_train_valnew['Year']).reshape(-1,1)
    Y99 = np.array(data_train_valnew['yield(t/ha)']).reshape(-1,1)
    #apply exec() method
    print(data_train_valnew,data_testnew)
    linear_m.fit(X99, Y99)
    print('state:',i)
    print('The foluma of de-trend the yield: yield_new = yield - {:.3f}×year + {:.3f} - ({:.3f})'
          .format(linear_m.coef_[0,0],yield_mean,linear_m.intercept_[0]))
    # de-trend data_train_val yield 
    data1 = data_train_valnew.copy()
    data1.loc[:,'yield(t/ha)_fix'] = data_train_valnew['yield(t/ha)'] - linear_m.coef_[0] * data_train_valnew['Year'] + yield_mean - linear_m.intercept_
    data1 = data1.drop(['yield(t/ha)'], axis = 1).rename({'yield(t/ha)_fix':'yield(t/ha)'}, axis=1)
    data_train_valnew = data1
    data_train_valnew1 = pd.concat([data_train_valnew1,data_train_valnew])
    # de-trend data_test yield 
    data1 = data_testnew.copy()
    data1.loc[:,'yield(t/ha)_fix'] = data_test['yield(t/ha)'] - linear_m.coef_[0] * data_test['Year'] + yield_mean - linear_m.intercept_

    data1 = data1.drop(['yield(t/ha)'], axis = 1).rename({'yield(t/ha)_fix':'yield(t/ha)'}, axis=1)
    data_testnew = data1
    data_testnew1 = pd.concat([data_testnew1,data_testnew])
data_test = data_testnew1
data_train_val = data_train_valnew1
y_train_val = data_train_val['yield(t/ha)']
y_test = data_test['yield(t/ha)']
X_train_val = data_train_val.drop(['sta_con','State','yield(t/ha)','Year','yield_N5_EXP2','yield_N5_EXP3','cluster','yield_N5_EXP4'], axis=1)
X_test = data_test.drop(['sta_con','State','yield(t/ha)','Year','yield_N5_EXP2','yield_N5_EXP3','cluster','yield_N5_EXP4'], axis=1)
X_train_val

In [ ]:
for a,b in zip(['A'],range(3)):
#     print(a,b)

    #apply exec() method
    exec("%s = None" % (a))
    print(eval(a))
#     #print string
#     print("output : ",A) 


In [ ]:
eval('x')=1
a

In [ ]:
y_test

In [ ]:
import numpy as np
from sklearn import linear_model
# 近邻线性趋势产量
def linear_yield_of_N_years(data,N_year = 30,N_nearest = 6):
    data['name'] = data.apply(
    lambda row: find_nearest(row['Latitude'], row['Longitude'],N_nearest), 
    axis=1)
    for county in set(data['sta_con']):
        for year in range(2004,2022):
            yearlist = [i for i in range(year-N_year,year)]
            countylist = data.loc[data['sta_con'] == county, 'name'].iloc[0]
            print(county,year,countylist)
            data1 = data[(data['Year'].isin(yearlist)) & (data['sta_con'].isin(countylist))][['Year','yield(t/ha)']]
            print(data1)
            if len(data1) == 1:
                data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N'+str(N_year)+'_EXP3'] = data.loc[(data['Year'].isin(yearlist)) & (data['sta_con']==county),'yield(t/ha)'].mean()
            elif len(data1) >1:
                linear_m = linear_model.LinearRegression()
                linear_m.fit(np.array(data1['Year']).reshape(-1,1),np.array(data1['yield(t/ha)']).reshape(-1,1))
                slope = linear_m.coef_[0]
                intercept = linear_m.intercept_
                print(slope,intercept)
                data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N'+str(N_year)+'_EXP3'] = slope*data.loc[(data['Year']==year) & (data['sta_con']==county),'Year']+intercept
        data = data.fillna(data.groupby(['Year','State']).transform('mean'))
    return data


%time linear_yield_of_N_years(data,5)[['Year','sta_con','yield(t/ha)','yield_N5_EXP2','yield_N5_EXP3']]

In [ ]:
data.loc[data['sta_con'] == 'illinois_de witt', 'name'].iloc[0]


In [ ]:
import numpy as np
from sklearn import linear_model
# 线性趋势产量
def linear_yield_of_N_years_nearest(data,N):
    for county in set(data['sta_con']):
        for year in range(2004,2022):
            yearlist = [i for i in range(year-N,year)]
            print(county,year)
            data1 = data[(data['Year'].isin(yearlist)) & (data['sta_con']==county)][['Year','yield(t/ha)']]
            print(data1)
            if len(data1) == 1:
                data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N'+str(N)+'_EXP3'] = data.loc[(data['Year'].isin(yearlist)) & (data['sta_con']==county),'yield(t/ha)'].mean()
            elif len(data1) >1:
                linear_m = linear_model.LinearRegression()
                linear_m.fit(np.array(data1['Year']).reshape(-1,1),np.array(data1['yield(t/ha)']).reshape(-1,1))
                slope = linear_m.coef_[0]
                intercept = linear_m.intercept_
                print(slope,intercept)
                data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N'+str(N)+'_EXP3'] = slope*data.loc[(data['Year']==year) & (data['sta_con']==county),'Year']+intercept
        data = data.fillna(data.groupby(['Year','State']).transform('mean'))
    return data


%time linear_yield_of_N_years(data,5)[['Year','sta_con','yield(t/ha)','yield_N5_EXP2','yield_N5_EXP3']]

In [ ]:
import math

math.ceil(len(set(data['sta_con']))/3)

In [ ]:
# 带聚类的线性趋势产量
# data = pd.DataFrame({'Year':[i for i in range(2012,2022)]+[2015],
#                      'State':['a','b','c','a','b','c','a','c','v','a','a'],
#                      'sta_con':['a','a','a','a','a','a','a','b','a','a','b'],
#                      'yield(t/ha)':[1,2,3,3,5,6,7,8,8.5,10,10],
#                      'cluster':[0,0,0,0,0,1,1,1,1,1,0]
#                         })

def linear_yield_of_N_years_cluster(data,N):
    for cluster in set(data['cluster']):
        for year in range(2004,2022):
            yearlist = [i for i in range(year-N,year)]
            print(cluster,year)
            data1 = data[(data['Year'].isin(yearlist)) & (data['cluster']==cluster)][['Year','yield(t/ha)']]
            print(data1)
            if len(data1) == 1:
                data.loc[(data['Year']==year) & (data['cluster']==cluster),'yield_N'+str(N)+'_EXP4'] = data.loc[(data['Year'].isin(yearlist)) & (data['cluster']==cluster),'yield(t/ha)'].mean()
            elif len(data1) >1:
                linear_m = linear_model.LinearRegression()
                linear_m.fit(np.array(data1['Year']).reshape(-1,1),np.array(data1['yield(t/ha)']).reshape(-1,1))
                slope = linear_m.coef_[0]
                intercept = linear_m.intercept_
                print(slope,intercept)
                data.loc[(data['Year']==year) & (data['cluster']==cluster),'yield_N'+str(N)+'_EXP4'] = slope*data.loc[(data['Year']==year) & (data['cluster']==cluster),'Year']+intercept
        data = data.fillna(data.groupby(['Year','State']).transform('mean'))
    return data
        
%time linear_yield_of_N_years_cluster(data,5)[['Year','sta_con','cluster','yield(t/ha)','yield_N5_EXP2','yield_N5_EXP4']]

In [ ]:
# 去趋势产量
def detrend_yield_of_N_years(data,N):
    for county in set(data['sta_con']):
        for year in range(2004,2022):
            yearlist = [i for i in range(year-N,year)]
            print(county,year)
            data1 = data[(data['Year'].isin(yearlist)) & (data['sta_con']==county)][['Year','yield(t/ha)']]
            print(data1)
            if len(data1) == 1:
                data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N'+str(N)+'_EXP3'] = data.loc[(data['Year'].isin(yearlist)) & (data['sta_con']==county),'yield(t/ha)'].mean()
            elif len(data1) >1:
                linear_m = linear_model.LinearRegression()
                linear_m.fit(np.array(data1['Year']).reshape(-1,1),np.array(data1['yield(t/ha)']).reshape(-1,1))
                slope = linear_m.coef_[0]
                intercept = linear_m.intercept_
                print(slope,intercept)
                data.loc[(data['Year']==year) & (data['sta_con']==county),'yield_N'+str(N)+'_EXP3'] = slope*data.loc[(data['Year']==year) & (data['sta_con']==county),'Year']+intercept
        data = data.fillna(data.groupby(['Year','State']).transform('mean'))
    return data


%time linear_yield_of_N_years(data,5)[['Year','sta_con','yield(t/ha)','yield_N5_EXP2','yield_N5_EXP3']

In [ ]:
data.at[(year, county), 'cluster']